# Project 1 — Data Warehouse ELT (DuckDB)

**Audience:** Technical team

This notebook demonstrates a practical ELT pattern with clear layers:

- **Raw**: land the data as-is (minimal transformation)
- **Staging**: apply data type casting + basic cleansing rules
- **Mart**: build a **star schema** (dimensions + fact) optimized for analytics

The code is intentionally function-driven with `print()` checkpoints for traceability during demos and debugging.

In [ ]:
import pandas as pd
import duckdb
import time
from pathlib import Path

## 0) Inputs
We use small sample CSVs located in `./data/`:
- `orders.csv`
- `customers.csv`

In a real production setup, these would come from object storage (S3/ADLS), a landing database, or a streaming/batch ingestion layer.

In [ ]:
def load_source_csvs(data_dir: str = "data"):
    """Load source CSVs and report basic shapes for observability."""
    print("[ELT] Loading source CSVs...")
    orders = pd.read_csv(Path(data_dir) / "orders.csv")
    customers = pd.read_csv(Path(data_dir) / "customers.csv")
    print(f"[ELT] orders shape: {orders.shape}")
    print(f"[ELT] customers shape: {customers.shape}")
    return orders, customers


In [ ]:
def connect_duckdb(db_path: str = "warehouse.duckdb"):
    """Create a DuckDB connection (local analytics DB)."""
    print(f"[ELT] Connecting to DuckDB: {db_path}")
    return duckdb.connect(db_path)

def ensure_schemas(con):
    """Create schemas to separate layers."""
    con.execute("CREATE SCHEMA IF NOT EXISTS raw;")
    con.execute("CREATE SCHEMA IF NOT EXISTS staging;")
    con.execute("CREATE SCHEMA IF NOT EXISTS mart;")
    print("[ELT] Schemas ready: raw, staging, mart")


## 1) Raw layer
We add a lightweight `ingested_at` timestamp to support lineage and auditability.

In [ ]:
def load_raw_layer(con, orders, customers):
    print("[ELT] Loading raw layer...")
    orders = orders.copy()
    orders["ingested_at"] = int(time.time())

    con.register("orders_df", orders)
    con.register("customers_df", customers)

    con.execute("CREATE OR REPLACE TABLE raw.orders AS SELECT * FROM orders_df;")
    con.execute("CREATE OR REPLACE TABLE raw.customers AS SELECT * FROM customers_df;")
    print("[ELT] Raw tables created: raw.orders, raw.customers")


## 2) Staging layer
Typical responsibilities:
- enforce types
- trim strings
- drop obviously invalid rows

You can extend with more rules (dedupe, standardization, referential checks, etc.).

In [ ]:
def build_staging_layer(con):
    print("[ELT] Building staging layer...")

    con.execute("""
    CREATE OR REPLACE TABLE staging.orders AS
    SELECT
      CAST(order_id AS BIGINT) AS order_id,
      CAST(customer_id AS BIGINT) AS customer_id,
      CAST(order_ts AS TIMESTAMP) AS order_ts,
      CAST(amount AS DOUBLE) AS amount,
      TRIM(region) AS region,
      TRIM(channel) AS channel,
      CAST(ingested_at AS BIGINT) AS ingested_at
    FROM raw.orders
    WHERE order_id IS NOT NULL AND customer_id IS NOT NULL;
    """)

    con.execute("""
    CREATE OR REPLACE TABLE staging.customers AS
    SELECT
      CAST(customer_id AS BIGINT) AS customer_id,
      TRIM(customer_name) AS customer_name,
      TRIM(industry) AS industry,
      TRIM(country) AS country
    FROM raw.customers
    WHERE customer_id IS NOT NULL;
    """)

    print("[ELT] Staging tables created: staging.orders, staging.customers")


## 3) Mart layer — Star schema
We create:
- `dim_customer`
- `dim_date`
- `fact_orders`

This structure is optimized for BI tools and SQL analytics.

In [ ]:
def build_mart_layer(con):
    print("[ELT] Building mart layer (star schema)...")

    con.execute("""
    CREATE OR REPLACE TABLE mart.dim_customer AS
    SELECT customer_id, customer_name, industry, country
    FROM staging.customers;
    """)

    con.execute("""
    CREATE OR REPLACE TABLE mart.dim_date AS
    SELECT DISTINCT
      DATE_TRUNC('day', order_ts)::DATE AS date_id,
      EXTRACT(year FROM order_ts)::INT AS year,
      EXTRACT(month FROM order_ts)::INT AS month,
      EXTRACT(day FROM order_ts)::INT AS day
    FROM staging.orders;
    """)

    con.execute("""
    CREATE OR REPLACE TABLE mart.fact_orders AS
    SELECT
      order_id,
      customer_id,
      DATE_TRUNC('day', order_ts)::DATE AS date_id,
      amount,
      region,
      channel,
      ingested_at
    FROM staging.orders;
    """)

    print("[ELT] Mart tables created: mart.dim_customer, mart.dim_date, mart.fact_orders")


## 4) Run pipeline + quick analytical query
We compute a simple revenue-by-industry aggregation to validate the mart usability.

In [ ]:
def run_pipeline():
    orders, customers = load_source_csvs()
    con = connect_duckdb("warehouse.duckdb")
    try:
        ensure_schemas(con)
        load_raw_layer(con, orders, customers)
        build_staging_layer(con)
        build_mart_layer(con)

        rows = con.execute("SELECT COUNT(*) FROM mart.fact_orders").fetchone()[0]
        print("[ELT] fact_orders row count:", rows)

        df = con.execute("""
            SELECT c.industry, SUM(f.amount) AS revenue
            FROM mart.fact_orders f
            JOIN mart.dim_customer c USING(customer_id)
            GROUP BY 1
            ORDER BY revenue DESC
        """).df()
        print("[ELT] Revenue by industry (sanity check):")
        display(df)
    finally:
        con.close()
        print("[ELT] Done.")

run_pipeline()
